In [ ]:
%load_ext autoreload
%autoreload

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
import re
from keras.layers import Input, Dense
from keras.models import Model
from keras.callbacks import EarlyStopping, TensorBoard

#plt.style.use('seaborn')
plt.style.use('ggplot')

Reads a subcategory of Amazon reviews into a pandas dataframe, and saves 2000 rows as a .pkl file.

Restarting Kernal while only running the following cell allows EDA without excessive memory use

In [ ]:
#Don't run if there is already a pickled pandas dataframe in 'data/small_df.pkl'
df = pd.read_csv('data/gaming_reviews.tsv', sep='\t', error_bad_lines=False)
df = df[['review_body', 'review_headline', 'star_rating']]
df = df[0:4000]
df.to_pickle('data/small_df.pkl')

In [ ]:
df = pd.read_pickle("data/small_df.pkl")
df.shape

The review_headline and review_body columns will both be useful for prediction.
Review date might be useful as well, such as day of the week reviewed. 

In [ ]:
#Creates a list with the number of ratings per star
rating_count_list = []
possible_ratings = [1, 2, 3, 4, 5]

for rating in possible_ratings:
    rating_count_list.append(df[df['star_rating'] == rating].shape[0])
    
#Creates array that stores fraction of the total star values
rating_proportion = np.array(rating_count_list)/sum(rating_count_list)

fig, ax = plt.subplots(figsize=(10, 5))
ax.bar(possible_ratings, rating_count_list)

ax.set_xlabel('Star rating')
ax.set_ylabel('Count')
ax.set_title('Number of ratings per star value')
ax.text(0.5, 2500, 'Star rating fraction of total', fontsize=20)
ax.text(0.5, 2300, '1: {:0.2f}'.format(rating_proportion[0]), fontsize=20)
ax.text(0.5, 2100, '2: {:0.2f}'.format(rating_proportion[1]), fontsize=20)
ax.text(0.5, 1900, '3: {:0.2f}'.format(rating_proportion[2]), fontsize=20)
ax.text(0.5, 1700, '4: {:0.2f}'.format(rating_proportion[3]), fontsize=20)
ax.text(0.5, 1500, '5: {:0.2f}'.format(rating_proportion[4]), fontsize=20)

In [ ]:
strip_chars = ".?,!;:\"'()" 
rgx = re.compile('[%s]' % strip_chars)

def process_str(row):

    body_list = []
    try:
        for word in row.lower().split(): 
            body_list.append(rgx.sub('', word))
        return body_list
    except Exception as e:
        print(e)
        return ['']

In [ ]:
#Creates a new column with the count vectorized strings in preparation for autoencoding. 
#Drops unneeded columns
df['text'] = df['review_headline'] + df['review_body']
cv = CountVectorizer(strip_accents='ascii')
sparse_vec = cv.fit_transform(df.text)
df['sparse'] = list(sparse_vec)
df = df[['text', 'sparse', 'star_rating']]

In [ ]:
df.sparse[0].shape

In [166]:
def autoencoder_model(X_train):
    '''
    defines autoencoder model
    input: X_train (2D np array)
    output: autoencoder (compiled autoencoder model)
    '''
    # this is our input placeholder
    input_img = Input(shape=(X_train[0].shape[1],))

    # first encoding layer
    encoded1 = Dense(units = 256, activation = 'relu')(input_img)

    # second encoding layer
    # note that each layer is multiplied by the layer before
    encoded2 = Dense(units = 64, activation='relu')(encoded1)

    # first decoding layer
    decoded1 = Dense(units = 256, activation='relu')(encoded2)

    # second decoding layer - this produces the output
    decoded2 = Dense(units = 784, activation='sigmoid')(decoded1)

    # this model maps an input to its reconstruction
    autoencoder = Model(input_img, decoded2)

    # compile model
    autoencoder.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics=['mse'])

    return autoencoder

In [ ]:
X_train = df.sparse[400:].reset_index(drop=True)
X_test = df.sparse[0:400].reset_index(drop=True)


In [ ]:
X = X_train
shuffle_index = np.arange(np.shape(X)[0])
np.random.shuffle(shuffle_index)
X =  X[shuffle_index]
#X = X.apply(lambda x: x.todense())



In [205]:
samples_per_epoch = 100
batch_size = 50
counter = 0
number_of_batches = samples_per_epoch/batch_size
shuffle_index = np.arange(np.shape(X)[0])
index_batch = shuffle_index[batch_size*counter:batch_size*(counter+1)]

X_batch = X[index_batch]
X_batch = X_batch.apply(lambda x: x.toarray()).reset_index(drop=True)

vocab_size = 10485

#np.array(X_batch).reshape(-1, vocab_size).shape
X_batch

0     [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...
1     [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...
2     [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...
3     [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...
4     [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...
5     [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...
6     [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...
7     [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...
8     [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...
9     [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...
10    [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...
11    [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...
12    [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...
13    [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...
14    [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...
15    [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...
16    [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...
17    [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [206]:
samples_per_epoch = 100

def batch_generator(X, batch_size):
    number_of_batches = samples_per_epoch/batch_size
    counter=0
    shuffle_index = np.arange(np.shape(X)[0])
    np.random.shuffle(shuffle_index)
    
    while 1:
        index_batch = shuffle_index[batch_size*counter:batch_size*(counter+1)]
        X_batch = X[index_batch]
        X_batch = X_batch.apply(lambda x: x.toarray()).reset_index(drop=True)
        X_batch = np.array(X_batch).reshape(-1,)
        counter += 1
        yield X_batch, X_batch
        break
        if (counter < number_of_batches):
            np.random.shuffle(shuffle_index)
            counter=0

In [ ]:
X_train[0].reshape(-1, 1).shape

In [211]:
for thang in batch_generator(X_train, 50):
    batch = thang
    break

for i, thang in enumerate(batch[0]):
    test = thang[0]
    print(test.shape)
    print(type(test))
    if i == 1:
        break
    

(10485,)
<class 'numpy.ndarray'>
(10485,)
<class 'numpy.ndarray'>


In [213]:
model = autoencoder_model(X_train)

batch_size = 100

# instantiate callbacks
tensorboard = TensorBoard(log_dir='./autoencoder_logs', histogram_freq=2, batch_size=batch_size, write_graph=True, write_grads=True, write_images=True)
earlystopping = EarlyStopping(monitor='val_loss', patience=2)

# try different number of epochs - 10 gives good performanace 
"""model.fit(X_train, X_train, epochs=10, batch_size=batch_size, verbose=1,
          validation_split=0.1, callbacks = [earlystopping, tensorboard])""" # cross val to estimate test error

nb_epoch = 10

model.fit_generator(generator=batch_generator(X_train, batch_size),
                    epochs=nb_epoch,
                    samples_per_epoch=10)


scores = model.evaluate(X_test, X_test)
print('Test mse = {}'.format(scores[0]))

X_test_decoded = model.predict(X_test)

plot_reconstruction(X_test, X_test_decoded)



/home/tyler/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(generator=<generator..., epochs=10, steps_per_epoch=10)`


Epoch 1/10


ValueError: Error when checking input: expected input_15 to have shape (10485,) but got array with shape (1,)